## Primer modelo
Vamos a tomar un conjunto de datos creados de manera sintética y utilizaremos el descenso del gradiente para optimizar un modelo que se ajuste a ellos.
Para el caso más sencillo utilizaremos una relación lineal entre las variables


Supondremos un modelo que depende de dos variables $f(x_1,x_2)=y$ con una relación lineal entre las variables de la manera 
$y=w_1x_1+w_2x_2+b$

Supondremos que partimos de un conjunto de n datos, los cuales llamaremos ejemplos: de la forma 
$$y_1=w_1x_1^1+w_2x_1^2+b$$
$$y_2=w_1x_2^1+w_2x_2^2+b$$
$$...$$
$$y_n=w_1x_n^1+w_2x_n^2+b$$

$$Y=XW+b$$

Con Y el vector que contiene a los $y_i$, X y W análogamente contienen a $w_i$ y a $x_i^j$

Para entrenar este modelo vamos a generar definir un par de $w_1$ y $w_2$ y les agregaremos ruido aleatorio

In [2]:
import numpy as np
import random

In [3]:
#trabajamos en el dominio [-10,10]
#Incluimos a b en w, w=[w1,w2,b]
def datos_prueba(w,n):
    #n será el número de ejemplos
    #generamos el dominio de los datos, aleatorios con distribución normal
    X=np.random.normal(-10,10,(n,len(w)-1))
    
    X1=[]
    for i in X:
        i=np.append(i,1.0)
        X1.append(i)
    X1=np.array(X1)
    Y = np.dot(X1, w) 
    Y=Y+np.random.normal(-1.0,0.8,Y.shape)
    return X1,Y



#### Minibatch
Con el fin de ahorrar costo computacional sólo trabajaremos con un porcentaje del número total de ejemplos, a este subconjunto se le llama minibatch

In [4]:
def minibatch(size,X,Y):
    n=len(X)
    #tomamos la lista de índices de tamaño n
    indices=np.arange(0,n)
    indices=list(indices)
    #Tomamos una lista aleatoria de índices de tamaño=size
    ibatch=random.sample(indices, size)
    Xbatch=[]
    Ybatch=[]
    #Generamos las nuevas listas random
    for i in ibatch:
        Xbatch.append(X[i])
        Ybatch.append(Y[i])
        
    return np.array(Xbatch),np.array(Ybatch)
    

##### Definiendo la función error
Para calcular el error en nuestro modelo de aproximación debemos calcular el error de aproximación por cada ejemplo y para el conjunto completo: la suma de estos.
Es decir la función de error para un conjunto de n ejemplos será:
$$Error=E(W,b)=\sum_{i=1}^{n}l_i(W,b)$$

Donde $l_i$ será el error de cada ejemplo, en este caso se tomará el error cuadrático medio.
Entonces:
$$l_i=\frac{1}{2}(\bar{y_i}-y_i)^2$$

Donde $\bar{y_i} será el valor estimado por nuestro modelo$
entonces para este caso la función error será 
$$l_i=\frac{1}{2}((w_1x_1^i+w_2x_2^i+b)-y_i)²$$

Y el error total
$$E(W,b)=\sum_{i=1}^{n}\frac{1}{2}((w_1x_1^i+w_2x_2^i+b)-y_i)²$$


Modificaremos las variables para meter b en W, agregamos b a w
y añadimos una columna de "1" a X


#### Aprendizaje
Con ayuda del descenso del gradiente podemos encontrar los valores que nos otorguen el modelo esperado

In [6]:
def descenso_grad(f,X0,eta):
   
    #definimos las derivadas parciales de una función de R^n a R
    #si X=(x1,x2,...,xk,...xn), debemos indicar con k respecto a cuál variable derivar
    def partial(g,k,X):
        h=1e-9
        Y=np.copy(X)
        X[k-1]=X[k-1]+h
        dp=(g(X)-g(Y))/h
        return dp
    #Ahora definimos la función que nos dará el gradiente
    def grad(f,X):
        grd=[]
        for i in np.arange(0,len(X)):
            ai=partial(f,i+1,X)
            grd.append(ai)
        return grd
    #Ahora se hacen las iteraciones
    i=0
    while True:
        i=i+1
        X0=X0-eta*np.array(grad(f,X0))
        if np.linalg.norm(grad(f,X0))<10e-8 or i>40: break
    return X0

In [13]:
w=[3,5,7] #w real de la forma [w1,w2,b] 
X1,Y1=datos_prueba(w,100)# datos sintéticos

def error(W):
    X,Y=minibatch(80,X1,Y1)
    #suma del error medio cuadrático de cada ejemplo
    s=0
    for i in range(0,len(Y)):
        l=0.5*(np.dot(X[i],W)-Y[i])**2
        s=s+l
    return s



In [17]:
descenso_grad(error,[1,1,1],0.1)

<ipython-input-13-028302cd328d>:9: RuntimeWarning: overflow encountered in double_scalars
  l=0.5*(np.dot(X[i],W)-Y[i])**2
<ipython-input-7-731a90d1388d>:9: RuntimeWarning: invalid value encountered in double_scalars
  dp=(g(X)-g(Y))/h


array([nan, nan, nan])

In [63]:
 def partial(g,k,X):
        h=1e-9
        Y=np.copy(X)
        X[k-1]=X[k-1]+h
        dp=(g(X)-g(Y))/h
        return dp
    #Ahora definimos la función que nos dará el gradiente
def grad(f,X):
    grd=[]
    for i in np.arange(0,len(X)):
        ai=partial(f,i+1,X)
        grd.append(ai)
    return grd

In [64]:
W=[1,1,1]
error(W)
#for i in range(1):
print(grad(error,W))
W=W-0.3*np.array(grad(error,W))
print(W)

[-76793.5998737812, -118223.89205917715, 5791.254807263613]
[23039.07996214 35468.20254235 -1736.34151758]


In [70]:
grad(error,[1,1,1])

[-76793.5998737812, -118223.89205917715, 5791.254807263613]